# Build Agentic Workflows with HuggingFace smolagents in Snowflake

## Overview

This guide outlines the process for creating agentic workflows in Snowflake Notebook on Container Runtime using [smolagents from Hugging Face](https://github.com/huggingface/smolagents). These agents are capable of writing Python code to call tools and orchestrate other agents. In this guide, we will also see how you can create a custom tool in **smolagent** that uses Snowflake Cortex.

## Step-By-Step Guide

For prerequisites and environment setup, please refer to the [QuickStart Guide](https://quickstarts.snowflake.com/guide/build-agentic-workflows-with-huggingface-smolagents-in-snowflake/index.html).

In [ ]:
# https://github.com/huggingface/smolagents

!pip install smolagents

In [ ]:
import streamlit as st
import pandas as pd
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, tool, LiteLLMModel, ToolCollection, Tool, ToolCallingAgent

# TODO: Replace hf_ZkEXVwIXXXXXXXXXXXXXXX with your Hugging Face token
model = HfApiModel(token="hf_ZkEXVwIXXXXXXXXXXXXXXX")

web_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool()],
    model=model,
    max_steps=10,
    name='web_search',
    description="Runs web searches for you. Give it your query as an argument.",
)

code_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[web_agent],
    additional_authorized_imports=['requests','json','pandas','matplotlib','xml.etree.ElementTree','bs4'],
)

In [ ]:
answer = 'N/A'
try:
    answer = code_agent.run("""
        Top 5 announcements at Snowflake Summit 2024 in JSON format. 
        Only return the JSON formatted output as the response and nothing else.
    """)
    parsed_answer = json.loads(answer)
    st.json(parsed_answer)
except:
    st.write(answer)

In [ ]:
answer = code_agent.run("""
        Top 5 blog articles on AI. Include blog title and link to the article. 
        Return the response in a Pandas dataframe and nothing else.
    """)
try:
    st.dataframe(pd.DataFrame(answer))
except:
    st.write(answer)

In [ ]:
class HFModelSnowflakeCortex(Tool):
    name="snowflake_cortex"
    description="""
    This tool summarizes text using Snowflake Cortex AI.
    """
    inputs = {
        "txt": {
            "type": "string",
            "description": "Text to analyze"
        }
    }
    output_type = "any"

    def __init__(self, cortex_llm):
        super().__init__(self)
        if cortex_llm is None:
            raise ValueError("""
                Missing Cortex LLM. Make sure you pass a valid LLM when creating an instance of HFModelSnowflakeCortex. 
                See https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions?_fsi=THrZMtDg,%20THrZMtDg&_fsi=THrZMtDg,%20THrZMtDg#availability.
            """) 
            
        self.cortex_llm = cortex_llm
        
    def forward(self,txt:str):
        import snowflake
        from snowflake.cortex import Complete
        response = {}
        prompt = f"""
                Summarize the text enclosed in ### in less than 200 words in JSON format and list out up to 3 highlights in JSON format ### {txt} ###. 
                Return only the JSON formatted output and nothing else.
        """
        response['summary'] = snowflake.cortex.Complete(self.cortex_llm,prompt)
        return response

In [ ]:
st.subheader("Streamlit Application")

text_to_summarize = """Snowflake ML is an integrated set of capabilities for end-to-end machine learning in a single platform on top of your governed data.

For out-of-the-box ML workflows in SQL, the ready-to-use ML Functions can help shorten development time and democratize ML across your organization. 
These functions let you train models for business use cases such as forecasting and anomaly detection without writing any code.

For custom ML workflows in Python, data scientists and ML engineers can easily and securely develop and productionize scalable features and models without 
any data movement, silos, or governance tradeoffs. The snowflake-ml-python library provides APIs for developing and deploying your Snowflake ML pipelines.

To build and operationalize models, data scientists and ML engineers can leverage a suite of Snowflake ML features. 
For model development, Snowflake ML Modeling APIs offer scalable data loading, feature engineering, and model training with distributed processing using 
CPUs or GPUs. For ML Operations (ML Ops), Snowflake ML includes the Feature Store and Model Registry for centralized management of features and models in production.

You can use Python APIs from the Snowpark ML library in Snowflake Notebooks, Snowsight worksheets. or your local Python IDE of choice."""

with st.container():
    with st.expander("Enter text and select LLM", expanded=True):
        
        with st.container():
            left_col,right_col = st.columns(2)
            with left_col:
                text_to_summarize = st.text_area('Text',text_to_summarize, height=550)
            with right_col:
                selected_llm = st.selectbox('Select LLM',('claude-3-5-sonnet','snowflake-llama-3.1-405b','llama3.1-405b'))

with st.container():
    _,mid_col,_ = st.columns([.4,.3,.3])
    with mid_col:
        generate_template = st.button('Generate summary and highlights ⚡',type="primary")

with st.container():
    if generate_template:
        with st.status("In progress...") as status:
            try:
                cortex_tool = HFModelSnowflakeCortex(selected_llm) 
                agent = CodeAgent(
                    tools=[cortex_tool],
                    model=model
                )
                answer = agent.run(text_to_summarize)
                parsed_answer = json.loads(answer)
                st.divider()
                st.subheader(f"Selected LLM: {selected_llm}")
                st.write("Final answer")
                st.json(parsed_answer)
            except:
                st.divider()
                st.subheader(f"Selected LLM: {selected_llm}")
                st.write("Final answer")
                st.write(answer)
            finally:
                status.update(label="Done!", state="complete", expanded=True)